
# ***1. Game Initialization***

pygame initializes the game.

Enum and namedtuple define movement directions and positions.

numpy is likely used for AI logic.

arial.ttf is used for text rendering in the game.



In [4]:
import pygame
import random
from enum import Enum
from collections import namedtuple
import numpy as np

pygame.init()
font = pygame.font.Font('/content/drive/MyDrive/Colab_Notebooks/arial.ttf', 25)  # Game font


# ***2. Defining Constants***

Direction Enum defines movement options.

Point is used for easy coordinate representation.

Colors are defined for game objects.

BLOCK_SIZE determines how far the snake moves in one step.



In [5]:
class Direction(Enum):
    RIGHT = 1
    LEFT = 2
    UP = 3
    DOWN = 4

Point = namedtuple('Point', 'x, y')

# Colors (RGB)
WHITE = (255, 255, 255)
RED = (200,0,0)  # Food
BLUE1 = (0, 0, 255)  # Snake body
BLUE2 = (0, 100, 255)
BLACK = (0,0,0)  # Background

BLOCK_SIZE = 20  # Each grid block is 20x20 pixels
SPEED = 40  # Speed of the game (higher = faster)


# ***3. SnakeGameAI Class***
This is the core class managing the game.

**a) Game Initialization**

Creates a Pygame window of size 640x480.

Sets the title to "Snake".

Calls reset() to initialize the game state.

In [6]:
class SnakeGameAI:

    def __init__(self, w=640, h=480):
        self.w = w
        self.h = h
        self.display = pygame.display.set_mode((self.w, self.h))
        pygame.display.set_caption('Snake')
        self.clock = pygame.time.Clock()
        self.reset()


**b) Resetting the Game**

Resets the snake's position to the center.

Sets initial direction to RIGHT.

Creates the snake's body (3 segments initially).

Places the first food item.

Resets score and frame_iteration, which track game progress.


In [7]:
    def reset(self):
      self.direction = Direction.RIGHT
      self.head = Point(self.w/2, self.h/2)
      self.snake = [
          self.head,
          Point(self.head.x - BLOCK_SIZE, self.head.y),
          Point(self.head.x - (2 * BLOCK_SIZE), self.head.y)
      ]
      self.score = 0
      self.food = None
      self._place_food()
      self.frame_iteration = 0


**c) Placing Food**

Generates a random (x, y) position for food.

Ensures food does not appear inside the snake.



In [8]:
    def _place_food(self):
      x = random.randint(0, (self.w - BLOCK_SIZE) // BLOCK_SIZE) * BLOCK_SIZE
      y = random.randint(0, (self.h - BLOCK_SIZE) // BLOCK_SIZE) * BLOCK_SIZE
      self.food = Point(x, y)
      if self.food in self.snake:
          self._place_food()


**d) Game Loop (Play Step)**

Increments frame_iteration (prevents infinite loops).

Moves the snake using _move(action).

Checks if the game is over (collision or too many iterations).

Rewards the snake for eating food (+10), punishes it for dying (-10).

Moves the snake forward (removes the tail unless it eats food).

Updates the UI and waits for the next frame.


In [9]:
    def play_step(self, action):
      self.frame_iteration += 1

      # Move the snake
      self._move(action)
      self.snake.insert(0, self.head)

      # Check if game over
      reward = 0
      game_over = False
      if self._is_collision() or self.frame_iteration > 100 * len(self.snake):
          game_over = True
          reward = -10
          return reward, game_over, self.score

      # Check if snake eats food
      if self.head == self.food:
          self.score += 1
          reward = 10
          self._place_food()
      else:
          self.snake.pop()

      # Update UI and clock
      self._update_ui()
      self.clock.tick(SPEED)

      return reward, game_over, self.score


**e) Collision Detection**

Detects if the snake hits the wall.

Detects if the snake collides with itself.


In [10]:
    def _is_collision(self, pt=None):
      if pt is None:
          pt = self.head
      if pt.x > self.w - BLOCK_SIZE or pt.x < 0 or pt.y > self.h - BLOCK_SIZE or pt.y < 0:
          return True
      if pt in self.snake[1:]:
          return True
      return False


**f) Moving the Snake**

Movement Logic

The AI inputs (action) are 3 values:

      [1, 0, 0] → Continue straight.

      [0, 1, 0] → Turn right.

      [0, 0, 1] → Turn left.

Determines the new direction based on the current one.

Moves the snake by updating its head position.


In [11]:
    def _move(self, action):
      clock_wise = [Direction.RIGHT, Direction.DOWN, Direction.LEFT, Direction.UP]
      idx = clock_wise.index(self.direction)

      if np.array_equal(action, [1, 0, 0]):  # No change
          new_dir = clock_wise[idx]
      elif np.array_equal(action, [0, 1, 0]):  # Right turn
          new_dir = clock_wise[(idx + 1) % 4]
      else:  # Left turn
          new_dir = clock_wise[(idx - 1) % 4]

      self.direction = new_dir

      x = self.head.x
      y = self.head.y
      if self.direction == Direction.RIGHT:
          x += BLOCK_SIZE
      elif self.direction == Direction.LEFT:
          x -= BLOCK_SIZE
      elif self.direction == Direction.DOWN:
          y += BLOCK_SIZE
      elif self.direction == Direction.UP:
          y -= BLOCK_SIZE

      self.head = Point(x, y)


**g) Updating UI**


Clears the screen.

Draws the snake in blue.

Draws the food in red.

Displays the score.



In [12]:
    def _update_ui(self):
      self.display.fill(BLACK)

      for pt in self.snake:
          pygame.draw.rect(self.display, BLUE1, pygame.Rect(pt.x, pt.y, BLOCK_SIZE, BLOCK_SIZE))
          pygame.draw.rect(self.display, BLUE2, pygame.Rect(pt.x+4, pt.y+4, 12, 12))

      pygame.draw.rect(self.display, RED, pygame.Rect(self.food.x, self.food.y, BLOCK_SIZE, BLOCK_SIZE))

      text = font.render("Score: " + str(self.score), True, WHITE)
      self.display.blit(text, [10, 10])
      pygame.display.flip()
